In [3]:
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt

# handle exception trace for debugging
# background loop
import traceback

import time
import random

In [4]:
cam = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice())
cam.Open()

# to get consistant results it is always good to start from "power-on" state
cam.UserSetSelector = "Default"
cam.UserSetLoad.Execute()

cam.ExposureTime = cam.ExposureTime.Min

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7144\2666066270.py:5: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.UserSetSelector.Value = Default
  cam.UserSetSelector = "Default"
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7144\2666066270.py:8: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 8.0
  cam.ExposureTime = cam.ExposureTime.Min


In [5]:
# show expected framerate max framerate ( @ minimum exposure time) in [µs]
cam.ResultingFrameRate.Value

56.45889792231256

In [4]:
# this results in frame period in µs in [fps]
1 / cam.ResultingFrameRate.Value * 1e6

6402.000000000001

we now compare different pylon grab strategies and their resulting frameperiod to this minimum frameperiod

# Grab scenarios

## GrabOne Loop

most simple style to grab

In [6]:
def GrabOneSample():
    # fetch some images with foreground loop
    img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.float32)

    for i in range(100):
        with cam.GrabOne(1000) as res:
            img = res.Array
            img_sum += img
    return img_sum

In [7]:
%%timeit -o
GrabOneSample()

6.23 s ± 8.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 6.23 s ± 8.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [8]:
grab_one_average = _.average / 100

In [9]:
f"time to capture one frame is: {grab_one_average*1e6:.2f} µs"

'time to capture one frame is: 62289.35 µs'

this is very easy to use, but will have the overhead of starting and stopping the grab engine for every frame

## Foreground Loop

move the start and stop of the grab engine out of the inner grab loop

In [10]:
def ForegroundLoopSample():
    # fetch some images with foreground loop
    img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)
    cam.StartGrabbingMax(100)
    while cam.IsGrabbing():
        with cam.RetrieveResult(1000) as res:
            if res.GrabSucceeded():
                img = res.Array
                img_sum += img
            else:
                raise RuntimeError("Grab failed")
    cam.StopGrabbing()
    return img_sum

In [11]:
%%timeit -o 
ForegroundLoopSample()

1.82 s ± 852 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 1.82 s ± 852 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [12]:
foreground_average = _.average / 100

In [13]:
f"average time to capture one frame is: {foreground_average*1e6:.2f} µs"

'average time to capture one frame is: 18233.29 µs'

## Background loop

* use this mode to communicate with the camera while the images are collected in the backgound.
* Allows async communication

Pylon uses classes with callback handler to let usercode communicate with the grab loop

In [14]:
class ImageHandler(py.ImageEventHandler):
    def __init__(self):
        super().__init__()
        self.img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)

    def OnImageGrabbed(self, camera, grabResult):
        """we get called on every image
        !! this code is run in a pylon thread context
        always wrap your code in the try .. except to capture
        errors inside the grabbing as this can't be properly reported from
        the background thread to the foreground python code
        """
        try:
            if grabResult.GrabSucceeded():
                # check image contents
                img = grabResult.Array
                self.img_sum += img
            else:
                raise RuntimeError("Grab Failed")
        except Exception as e:
            traceback.print_exc()

In [15]:
def BackGroundLoopSample():

    # instantiate callback handler
    handler = ImageHandler()
    # register with the pylon loop
    cam.RegisterImageEventHandler(
        handler, py.RegistrationMode_ReplaceAll, py.Cleanup_None
    )

    # fetch some images with background loop
    cam.StartGrabbingMax(
        100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera
    )
    while cam.IsGrabbing():
        # random exposuretime changes every 100ms
        cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
        time.sleep(0.1)

    cam.StopGrabbing()
    cam.DeregisterImageEventHandler(handler)

    return handler.img_sum

In [16]:
%%timeit -o
BackGroundLoopSample()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_7144\3266499962.py:16: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 956.6554849206331
  cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7144\3266499962.py:16: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 972.0825095287254
  cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7144\3266499962.py:16: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 583.1374858401181
  cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_7144\3266499962.py:16: DeprecationWarning: Setting a feature value by direct assignment is deprecated. Use <nodemap>.ExposureTime.Value = 930.1608183494219
  cam.Ex

1.83 s ± 783 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 1.83 s ± 783 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [17]:
background_average = _.average / 100

In [18]:
f"average time to capture one frame is: {background_average*1e6:.2f} µs"

'average time to capture one frame is: 18341.14 µs'

In [19]:
cam.StopGrabbing()

In [20]:
cam.Close()